In [1]:
import MetaTrader5 as mt5
from datetime import datetime

from secret_login import Secret_Login

In [2]:
mt5.initialize()

True

## Login


In [3]:
temp = Secret_Login()

account = temp.login_key
password = temp.password
server = temp.server

print(f"Account : {account}")
print(f"password : {password}")
print(f"Server : {server}")



Account : 10002149264
password : 3BV82Vxw
Server : MetaQuotes-Demo


In [6]:
mt5.login(account, password, server)

False

In [5]:
mt5.account_info()

AccountInfo(login=10002149264, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=False, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10000.0, credit=0.0, profit=0.0, equity=10000.0, margin=0.0, margin_free=10000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='MetaTrader 5 Desktop Demo', server='MetaQuotes-Demo', currency='USD', company='MetaQuotes Software Corp.')